In [ ]:
from kiteconnect import KiteConnect, KiteTicker
import datetime as dt
import pprint
import pandas as pd
import time
from prettytable import PrettyTable
import winsound
from IPython.display import clear_output
import pickle
from cryptography.fernet import Fernet
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(lineno)d | %(message)s',
    handlers=[
        logging.FileHandler("logs/{}.log".format(dt.date.today().strftime("%Y_%m_%d"))),
        logging.StreamHandler()
    ]
)
log = logging.getLogger()

In [2]:
def logDaysPnL():
    tempdf = pd.DataFrame(kite.positions()['day'])
    tempdf = tempdf[['tradingsymbol', 'day_buy_quantity', 'day_buy_price', 'day_sell_quantity', 'day_sell_price', 'pnl']]
    table = PrettyTable()
    table.field_names = tempdf.columns.tolist()
    for row in tempdf.itertuples(index=False):
        table.add_row(row)
    log.info("\n"+table.get_string())
    

In [3]:
with open("syek_ogla.key", "rb") as key_file:
    key = key_file.read()
cipher = Fernet(key)
with open("syek_ogla.bin", "rb") as file:
    kite_credentials = pickle.loads(cipher.decrypt(file.read()))

In [4]:
kite = KiteConnect(api_key= kite_credentials[0])
print(kite.login_url())

https://kite.zerodha.com/connect/login?api_key=onrrp2354dne35g4&v=3


In [5]:
data = kite.generate_session("7z4rJFmsJMHsZKXQNqgs8mYTk9r0y7gs", api_secret=kite_credentials[1])
kite.set_access_token(data["access_token"])
log.info("logged into session")
pprint.pprint(kite.profile())

2024-11-29 09:48:25,321 | 3 | logged into session


{'avatar_url': None,
 'broker': 'ZERODHA',
 'email': '93prithvi@gmail.com',
 'exchanges': ['MF', 'BSE', 'NSE', 'BFO', 'NFO', 'BCD'],
 'meta': {'demat_consent': 'physical'},
 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'],
 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'],
 'user_id': 'XF6031',
 'user_name': 'Prithvi Ram Grandhi',
 'user_shortname': 'Prithvi',
 'user_type': 'individual/ind_with_nom'}


In [6]:
try:
    zerodha_orders = pd.DataFrame(kite.orders())
    print(zerodha_orders[['exchange_timestamp', 'tradingsymbol', 'transaction_type','quantity']])
except KeyError:
    raise("!!!!!!!!!!!!!!no orders are placed, please stop executing remaining cells!!!!!!!!!")
except Exception as e:
    print(f"An error occurred: {e}")


   exchange_timestamp      tradingsymbol transaction_type  quantity
0 2024-11-29 09:49:08  NIFTY24D0524050PE             SELL       100
1 2024-11-29 09:49:08  NIFTY24D0524050CE             SELL       100


In [9]:
putSymbol = "NIFTY24D0524050PE"#"BANKNIFTY2430647400PE"
callSymbol = "NIFTY24D0524050CE"
future = "NIFTY24DECFUT"#"NIFTY24NOVFUT"#"BANKNIFTY24NOVFUT"#"FINNIFTY24NOVFUT"#"MIDCPNIFTY24NOVFUT"
qty = 100
SL_JUMP_LIMIT = 5
oneLegSL = 10
lowerBandwidth = 20
log.info("---------------------------------------------------------------------------------")
log.info(f"{future} is selected, with qty of {qty} for strikes {putSymbol} and {callSymbol}")

2024-11-29 09:49:40,358 | 8 | ---------------------------------------------------------------------------------
2024-11-29 09:49:40,360 | 9 | NIFTY24DECFUT is selected, with qty of 100 for strikes NIFTY24D0524050PE and NIFTY24D0524050CE


In [10]:
callPrice = kite.ltp("NFO:"+callSymbol)["NFO:"+callSymbol]['last_price']
putPrice = kite.ltp("NFO:"+putSymbol)["NFO:"+putSymbol]['last_price']
with open('threshold.txt') as f:
    threshold = float(f.readlines()[0])
log.info("---------------------------------------------------------------------------------")    
log.info(f"{callSymbol}({callPrice}) + {putSymbol}({putPrice}) = {callPrice + putPrice}")
log.info(f"current set threshold is {threshold}")
log.info(f"current {future} price is {kite.ltp('NFO:'+future)['NFO:'+future]['last_price']}")

2024-11-29 09:49:41,422 | 5 | ---------------------------------------------------------------------------------
2024-11-29 09:49:41,422 | 6 | NIFTY24D0524050CE(210.8) + NIFTY24D0524050PE(182.55) = 393.35
2024-11-29 09:49:41,422 | 7 | current set threshold is 72.85
2024-11-29 09:49:41,525 | 8 | current NIFTY24DECFUT price is 24190


In [11]:
exitedCall, exitedPut= False, False
def placeOrder(tradeSymbol, qty, transactionType):
    try:
        kite.place_order(variety=kite.VARIETY_REGULAR, exchange=kite.EXCHANGE_NFO, tradingsymbol=tradeSymbol,
                    transaction_type=transactionType, quantity=qty, product=kite.PRODUCT_NRML, order_type=kite.ORDER_TYPE_MARKET,
                    price=None, validity=None, disclosed_quantity=None, trigger_price=None, tag = "vscode")
        winsound.Beep(frequency=2000, duration=500)
        print(f"current {tradeSymbol} price is {kite.ltp('NFO:'+tradeSymbol)['NFO:'+tradeSymbol]['last_price']}")
        log.info("---------------------------------------------------------------------------------")
        log.info(f"current {tradeSymbol} price is {kite.ltp('NFO:'+tradeSymbol)['NFO:'+tradeSymbol]['last_price']}")
        print(f"{qty} qty {transactionType} order placed for symbol: {tradeSymbol} at {dt.datetime.now().strftime('%H:%M:%S')}")
        log.info(f"{qty} qty {transactionType} order placed for symbol: {tradeSymbol} at {dt.datetime.now().strftime('%H:%M:%S')}")
    except Exception as e:
        winsound.Beep(frequency=1000, duration=200)
        winsound.Beep(frequency=1000, duration=200)
        log.info("---------------------------------------------------------------------------------")
        print(f"an error occured, please check manually : {e}")
        log.info(f"an error occured, please check manually : {e}")

In [12]:
while True:
    callPrice = 0 if exitedCall else kite.ltp("NFO:"+callSymbol)["NFO:"+callSymbol]['last_price']
    putPrice = 0 if exitedPut else kite.ltp("NFO:"+putSymbol)["NFO:"+putSymbol]['last_price']
    with open('threshold.txt') as f:
        threshold = float(f.readlines()[0])
    condition1 = (callPrice + putPrice) > threshold
    condition1 = condition1 and (0 <= dt.datetime.now().second < 10)
    condition2 = (callPrice + putPrice) > (SL_JUMP_LIMIT + threshold)
    condition3 = condition1 or condition2
    if condition3:
        if condition2:
            log.info("---------------------------------------------------------------------------------")
            log.info("##################### sudden spike in price be carefull !!! #####################")
            log.info("##################### sudden spike in price be carefull !!! #####################")
            log.info("##################### sudden spike in price be carefull !!! #####################")
        log.info("---------------------------------------------------------------------------------")
        log.info(f"call price({callPrice}) + put price({putPrice}) = {callPrice + putPrice}")
        log.info(f"current set threshold is {threshold}")
        log.info(f"current {future} price is {kite.ltp('NFO:'+future)['NFO:'+future]['last_price']}")
        clear_output(wait=True)
        if exitedCall:
            placeOrder(putSymbol, qty, kite.TRANSACTION_TYPE_BUY)
            exitedPut = True
            log.info("---------------------------------------------------------------------------------")
            log.info(f"Trading ended thank you!!!!")
            logDaysPnL()
            break

        elif exitedPut:
            placeOrder(callSymbol, qty, kite.TRANSACTION_TYPE_BUY)
            exitedCall = True
            log.info("---------------------------------------------------------------------------------")
            log.info(f"Trading ended thank you!!!!")
            logDaysPnL()
            break

        else:
            if callPrice > putPrice:
                placeOrder(callSymbol, qty, kite.TRANSACTION_TYPE_BUY)
                with open('threshold.txt', 'w') as f:
                    f.write(str(float(putPrice) + oneLegSL))
                exitedCall = True
                lowerBandwidth = lowerBandwidth/2
                log.info("---------------------------------------------------------------------------------")
                log.info(f"exited call leg - {callSymbol}")
                log.info(f"changed the put leg threshold to putPrice({float(putPrice)}) + oneLegSL({oneLegSL}) = {float(putPrice) + oneLegSL}")
                log.info(f"changed the lowerBandwidth to {lowerBandwidth}")
            else:
                placeOrder(putSymbol, qty, kite.TRANSACTION_TYPE_BUY)
                with open('threshold.txt', 'w') as f:
                    f.write(str(float(callPrice) + oneLegSL))
                exitedPut = True  
                lowerBandwidth = lowerBandwidth/2
                log.info("---------------------------------------------------------------------------------")
                log.info(f"exited put leg - {callSymbol}")
                log.info(f"changed the call leg threshold to callPrice({float(callPrice)}) + oneLegSL({oneLegSL}) = {float(callPrice) + oneLegSL}")
                log.info(f"changed the lowerBandwidth to {lowerBandwidth}")
        time.sleep(10)  
    elif ((callPrice + putPrice) < (threshold-lowerBandwidth)) and (0 <= dt.datetime.now().second < 10):
        log.info("---------------------------------------------------------------------------------")
        log.info(f"call price {callPrice} + put price {putPrice} = {callPrice + putPrice}")
        log.info(f"(lower limit) set threshold({threshold}) - lowerBandwidth({lowerBandwidth}) is {threshold-lowerBandwidth}")
        log.info(f"current {future} price is {kite.ltp('NFO:'+future)['NFO:'+future]['last_price']}")
        with open('threshold.txt', 'w') as f:
            f.write(str(float((callPrice + putPrice)+(lowerBandwidth/2))))      
        log.info(f"(upper limit) threshold to new value - callPrice({callPrice}) + putPrice({putPrice}) + LowerBandwidth({lowerBandwidth})/2 = {float((callPrice + putPrice)+(lowerBandwidth/2))}") 
        time.sleep(4)
    else:
        clear_output(wait=True)
        print(f"call price {callPrice} + put price {putPrice} = {callPrice + putPrice}")
        print(f"current set threshold is {threshold}")
        print(f"current lowerBandwidth price is {threshold-lowerBandwidth}")
        print(f"current {future} price is {kite.ltp('NFO:'+future)['NFO:'+future]['last_price']}")
        time.sleep(4)

2024-11-29 10:45:01,877 | 9 | ---------------------------------------------------------------------------------
2024-11-29 10:45:01,993 | 10 | current NIFTY24D0524050PE price is 159.85
2024-11-29 10:45:01,994 | 12 | 100 qty BUY order placed for symbol: NIFTY24D0524050PE at 10:45:01
2024-11-29 10:45:01,994 | 24 | ---------------------------------------------------------------------------------
2024-11-29 10:45:01,995 | 25 | Trading ended thank you!!!!


current NIFTY24D0524050PE price is 159.85
100 qty BUY order placed for symbol: NIFTY24D0524050PE at 10:45:01


2024-11-29 10:45:02,107 | 8 | 
+-------------------+------------------+---------------+-------------------+----------------+-------+
|   tradingsymbol   | day_buy_quantity | day_buy_price | day_sell_quantity | day_sell_price |  pnl  |
+-------------------+------------------+---------------+-------------------+----------------+-------+
| NIFTY24D0524050CE |       100        |     258.25    |        100        |     204.3      | -5395 |
| NIFTY24D0524050PE |       100        |     160.15    |        100        |     188.15     |  2800 |
+-------------------+------------------+---------------+-------------------+----------------+-------+


In [11]:
logDaysPnL()

2024-11-19 10:25:50,160 | 8 | 
+-------------------+------------------+---------------+-------------------+----------------+-------+
|   tradingsymbol   | day_buy_quantity | day_buy_price | day_sell_quantity | day_sell_price |  pnl  |
+-------------------+------------------+---------------+-------------------+----------------+-------+
| NIFTY24N2123650CE |       100        |     149.0     |        100        |     116.2      | -3280 |
| NIFTY24N2123650PE |       100        |      62.6     |        100        |     103.65     |  4105 |
+-------------------+------------------+---------------+-------------------+----------------+-------+


In [25]:
kite.quote("NFO:FINNIFTY24OCT24000PE")["NFO:FINNIFTY24OCT24000PE"]['depth']

{'buy': [{'price': 95.55, 'quantity': 250, 'orders': 5},
  {'price': 95.5, 'quantity': 425, 'orders': 4},
  {'price': 95.45, 'quantity': 675, 'orders': 6},
  {'price': 95.4, 'quantity': 450, 'orders': 5},
  {'price': 95.35, 'quantity': 275, 'orders': 5}],
 'sell': [{'price': 95.7, 'quantity': 25, 'orders': 1},
  {'price': 95.9, 'quantity': 50, 'orders': 1},
  {'price': 95.95, 'quantity': 375, 'orders': 5},
  {'price': 96, 'quantity': 475, 'orders': 3},
  {'price': 96.05, 'quantity': 425, 'orders': 4}]}

In [30]:
while True:
    currStraddlePrice = kite.ltp("NFO:"+putSymbol)["NFO:"+putSymbol]['last_price'] + kite.ltp("NFO:"+callSymbol)["NFO:"+callSymbol]['last_price']
    if (currStraddlePrice > threshold) and (0 <= dt.datetime.now().second < 5):
        order1 = kite.place_order(variety=kite.VARIETY_REGULAR, exchange=kite.EXCHANGE_NFO, tradingsymbol=putSymbol, 
                                  transaction_type=kite.TRANSACTION_TYPE_BUY, quantity=qty, product=kite.PRODUCT_NRML, order_type=kite.ORDER_TYPE_MARKET, 
                                  price=None, validity=None, disclosed_quantity=None, trigger_price=None, tag = "vscode")
        order2 = kite.place_order(variety=kite.VARIETY_REGULAR, exchange=kite.EXCHANGE_NFO, tradingsymbol=callSymbol,
                                  transaction_type=kite.TRANSACTION_TYPE_BUY, quantity=qty, product=kite.PRODUCT_NRML, order_type=kite.ORDER_TYPE_MARKET,
                                  price=None, validity=None, disclosed_quantity=None, trigger_price=None, tag = "vscode")
        winsound.Beep(frequency=2000, duration=500)
        clear_output(wait=True)
        print("Current straddle: ", currStraddlePrice)
        print("current time: ", dt.datetime.now())
        print("trading ended, Thanks!!!")
        print(order1)
        print(order2)
        break
    else:
        clear_output(wait=True)
        print("Current straddle: ", currStraddlePrice)
        print("threshold: ",threshold)
        print("Future: ", kite.ltp("NFO:"+future)["NFO:"+future]['last_price'])
        time.sleep(5)

Current straddle:  111.0
trading ended, Thanks!!!
240718201421013
240718201421045


In [38]:
import pickle
from cryptography.fernet import Fernet
key = Fernet.generate_key()
cipher = Fernet(key)
algo_keys = ['onre35g4', '0ftdvcqe3m']
encrypted_data = cipher.encrypt(pickle.dumps(algo_keys))
with open("syek_ogla.bin","wb") as file:
    file.write(encrypted_data)
with open("syek_ogla.key", "wb") as keyfile:
    keyfile.write(key)